# Explore PhotoZ from ECDFS_comcam_euclid

- Author: Sylvie Dagoret-Campagne
- Creation date : 2025-04-15
- Last update : 2025-04-23

For the photo-z science unit 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import lsst.afw.display as afwDisplay
import matplotlib.pyplot as plt
import numpy as np

import lsst
import lsst.geom as geom
from lsst.geom import Angle
from lsst.geom import SpherePoint
from lsst.geom import AngleUnit,degrees


## My tools

In [ ]:
def get_list_of_objmethods(my_object):
    """
    Rreturn the methods of an object
    """
    
    all_attributs_methods = dir(my_object)
    sel_attributs_methods = [meth for meth in all_attributs_methods if "_" not in meth]
    return sel_attributs_methods


In [ ]:
def getdocstring_from_objmethod(my_object,methodName):
    """
    Return the docstring aof a method in an object
    """
    docstring = getattr(my_object, methodName).__doc__
    return docstring

## Firefly.

In [ ]:
#afwDisplay.setDefaultBackend('firefly')
#afw_display = afwDisplay.Display(frame=1)

## Registry and data in the user collection

- Today 2025-04-15 from Eric Charles:

Ok.  I'm re-running the com_cam stuff.    
The new version will land in `/repo/main:u/echarles/pz_rail_ECDFS/com_cam_euclid/w_2025_10/v2` in about 20' or so.    
The input models are in /repo/main:u/echarles/pz_models/com_cam_euclid/v2 .
it would be great if:
People had a look at these:
At least 1-2 other people learned how to do this.

## Required installation of DM with Rail

    source /sdf/data/rubin/shared/pz/software/lsst-stack/loadLSST.sh
    setup lsst_distrib
    setup -r /sdf/data/rubin/shared/pz/software/packages/meas_pz -t ${USER}


Then install 

     pip install pz-rail-base
     pip install pz-rail-XX (for each of the algorithms I wanted to use)


## Logging on jupyterlab 

     - choose w_2025_10
     - put 
            setup -r /sdf/data/rubin/shared/pz/software/packages/meas_pz -t ${USER} 
            in notebooks/.user_setup

## Chech the environnement

In [ ]:
! eups list -s 

## PZ and E-Charles collection

In [ ]:
the_collection = 'u/echarles/pz_rail_ECDFS/com_cam_euclid/w_2025_10/v2'
instrument = "LSSTComCam"
skymapName = "lsst_cells_v1"
where_clause = "instrument = \'" + instrument+ "\'"
TRACTSEL = 5063

ECDFS_RA = 53.1
ECDFS_DEC = -27.8
ECDFS_COORD = SpherePoint(ECDFS_RA, ECDFS_DEC , degrees)

In [ ]:
from lsst.daf.butler import Butler
butler = Butler.from_config('/repo/main', collections=[the_collection])
registry = butler.registry

In [ ]:
skymap = butler.get('skyMap', skymap=skymapName, collections=the_collection)

In [ ]:
# 4. Trouver dans quel tract/patch se trouve la coordonnée
ECDFS_tract_info = skymap.findTract(ECDFS_COORD)

### Check the datasettypes

In [ ]:
if 0:
    for datasetType in registry.queryDatasetTypes():
        if registry.queryDatasets(datasetType, collections=the_collection):
            # Limit search results to the data products
            if ("pz" in datasetType.name) :
                print(datasetType)

## Retrieve the list of patches in the ECDFS

In [ ]:
#datasettype = "objectTable_tract"
#therefs = butler.registry.queryDatasets(datasettype,  collections = the_collection)

## Retrieve results of PhotoZ algo

- We know that we have 58 patches in tract 5063 (ECDFS):

      butler query-datasets /repo/main --collections u/echarles/pz_rail_ECDFS/com_cam_euclid/w_2025_10/v2 "pz_estimate*"
       
- need to skip patchs 9,19,29,39,40,49,50,51,52 and all after 57
  

In [ ]:
qp_ens = {}
NPATCHMAX = 58
patch_to_skip = np.array([9,19,29,39,40,49,50,51,52]) 
for ipatch in range(NPATCHMAX):
    if ipatch not in patch_to_skip:
        try:
            qp_ens_knn = butler.get('pz_estimate_knn', instrument="LSSTComCam", skymap="lsst_cells_v1", tract=TRACTSEL, patch=ipatch)
            qp_ens_bpz = butler.get('pz_estimate_bpz', instrument="LSSTComCam", skymap="lsst_cells_v1", tract=TRACTSEL, patch=ipatch)
            qp_ens_dnf = butler.get('pz_estimate_dnf', instrument="LSSTComCam", skymap="lsst_cells_v1", tract=TRACTSEL, patch=ipatch)
            qp_ens_fzboost = butler.get('pz_estimate_fzboost', instrument="LSSTComCam", skymap="lsst_cells_v1", tract=TRACTSEL, patch=ipatch)
            qp_ens_gpz = butler.get('pz_estimate_gpz', instrument="LSSTComCam", skymap="lsst_cells_v1", tract=TRACTSEL, patch=ipatch)
            #qp_ens_tpz = butler.get('pz_estimate_tpz', instrument="LSSTComCam", skymap="lsst_cells_v1", tract=TRACTSEL, patch=0)
            #qp_ens_lephare = butler.get('pz_estimate_lephare', instrument="LSSTComCam", skymap="lsst_cells_v1", tract=TRACTSEL, patch=0)
            qp_ens[ipatch] = {
                "qp_ens_knn": qp_ens_knn,
                "qp_ens_bpz": qp_ens_bpz,
                "qp_ens_dnf" : qp_ens_dnf,
                "qp_ens_fzboost" : qp_ens_fzboost,
                "qp_ens_gpz" : qp_ens_gpz
            }
        except Exception as inst:
            print(type(inst))    # the exception type
            print(inst.args)     # arguments stored in .args
            print(inst)          # __str__ allows args to be printed directly,
    

In [ ]:
NPATCHS = len(qp_ens)

In [ ]:
palette_patchs = sns.color_palette("husl", NPATCHMAX)
from matplotlib.colors import ListedColormap
cmap_patchs = ListedColormap(palette_patchs)
#plt.scatter(x, y, c=labels, cmap=cmap)

In [ ]:
palette_patchs 

In [ ]:
sns.palplot(palette_patchs)

In [ ]:
palette_patchs[25]

### Check what can be called from qp

https://qp.readthedocs.io/en/main/

In [ ]:
my_object = qp_ens_knn
sel_attributs_methods =  get_list_of_objmethods(my_object)
print(sel_attributs_methods)

In [ ]:
#for methodName in sel_attributs_methods:
#    docstring =  getdocstring_from_objmethod(my_object,methodName)
#    print(f"Method: {methodName}\nDocstring: {docstring}\n")

In [ ]:
list_of_algos = list(qp_ens[0].keys())
Nalgos = len(list_of_algos)
list_of_algos

In [ ]:
list_of_patches = list(qp_ens.keys())

In [ ]:
grid = np.linspace(0, 3, 101)
the_algo_name = list_of_algos[0]
the_patch_num = 10
the_qp_algo = qp_ens[the_patch_num][the_algo_name]
title = f"pdf patch = {the_patch_num}, algo = {the_algo_name}"
#print(the_qp_algo.mode(grid).flatten())

N=10
fig,axs = plt.subplots(N,1,figsize=(8,8),layout="constrained")
for idx in range(N):
    ax =axs[idx]
    if idx == 0:
        ax.set_title(title)
    data = the_qp_algo[idx].pdf(grid)
    label = f"obj = {idx}"
    ax.plot(grid,data[0,:],label=label)
    ax.legend()

ax.set_xlabel("z-grid")
  

In [ ]:
assert False

In [ ]:
assert False

In [ ]:
# loop on patches
grid = np.linspace(0, 3, 51)
for ipatch in list_of_patches:
    if ipatch < NPATCHMAX :
        the_qp = qp_ens[ipatch]
        print(ipatch,the_qp.keys())
        for idx,the_algo in enumerate(the_qp.keys()): 
            the_qp_algo = the_qp[the_algo]
            nobj = the_qp_algo.npdf
            the_modes = the_qp_algo.mode(grid).flatten()
            print(ipatch,the_algo,nobj,the_modes)

In [ ]:
assert False

In [ ]:
fig,axs = plt.subplots(5,1,figsize=(12,10))

# loop on algo
for ialgo in range(Nalgos):
    ax = axs[ialgo]
    the_algo = list_of_algos[ialgo]

    # loop on patches
    for ipatch in list_of_patches:
        if ipatch < 2:
            the_qp = qp_ens[ipatch][the_algo]
            nobj = the_qp.npdf
            if nobj>0:
                the_modes = the_qp.mode(grid).flatten()
                ax.hist(the_modes,bins=50,range=(0.,3.),histtype="step",color=palette_patchs[ipatch])
        #print(the_algo,ipatch,nobj,the_modes)

In [ ]:
assert False

In [ ]:
fig,axs = plt.subplots(5,1,figsize=(12,10))

# loop on algo
for ialgo in range(Nalgos):
    ax = axs[ialgo]
    the_algo = list_of_algos[ialgo]

    # loop on patches
    for ipatch in list_of_patches:
        if ipatch < 10:
            the_qp = qp_ens[ipatch][the_algo]
            nobj = the_qp.npdf
            if nobj>0:
                the_modes = the_qp.mode(grid).flatten()
                ax.hist(the_modes,bins=50,range=(0.,3.),histtype="step",color=palette_patchs[ipatch])
        #print(the_algo,ipatch,nobj,the_modes)
    
    

In [ ]:
objdata = qp_ens_knn.objdata()
objdata

In [ ]:
qp_ens_knn.npdf

In [ ]:
qp_ens_bpz.npdf

In [ ]:
qp_ens_dnf.npdf

In [ ]:

plt.hist(my_object.mode(grid).flatten(),bins=50);